In [2]:
# Import Libraries

import pandas as pd
import numpy as np

# Data Preprocessing

- **Data Cleaning:** Dealing with `null` values
- **Categorical Columns Processing:** Converting all the non-numerical(categorical, dates) to numerical values

In [9]:
np.nan == np.nan

False

In [10]:
# TODO: Import from utils
def define_user_profile(loan_status):
    # Define user profile based on loan status
    if loan_status in ('Fully Paid', 'Current'):
        return 'Payer'
    elif loan_status == loan_status:
        return 'Defaulter'
    else:
        return np.nan

In [11]:
df = pd.read_csv('../data/accepted_2007_to_2018Q4.csv')
df['user_profile']  = df['loan_status'].apply(define_user_profile)
df.drop(['loan_status'], axis=1, inplace=True)
df.head()

/var/folders/3z/1gg3czrd2ps4jtgnrk_j_nqmgw5xwd/T/ipykernel_81755/2454511268.py:1: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/accepted_2007_to_2018Q4.csv')


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,user_profile
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,Payer
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,Payer
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,Payer
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,Payer
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,Payer


In [25]:
# Columns selected from EDA
selected_cols = ['loan_amnt', 'int_rate', 'installment', 'issue_d', 'grade', 'sub_grade', 
                 'purpose', 'debt_settlement_flag', 'earliest_cr_line',
                 'home_ownership', 'fico_range_low', 'fico_range_high', 'dti']

df_selected = df[selected_cols + ['user_profile']]
df_selected.head()

,loan_amnt,int_rate,installment,issue_d,grade,sub_grade,purpose,debt_settlement_flag,earliest_cr_line,home_ownership,fico_range_low,fico_range_high,dti,user_profile
0,3600.0,13.99,123.03,Dec-2015,C,C4,debt_consolidation,N,Aug-2003,MORTGAGE,675.0,679.0,5.91,Payer
1,24700.0,11.99,820.28,Dec-2015,C,C1,small_business,N,Dec-1999,MORTGAGE,715.0,719.0,16.06,Payer
2,20000.0,10.78,432.66,Dec-2015,B,B4,home_improvement,N,Aug-2000,MORTGAGE,695.0,699.0,10.78,Payer
3,35000.0,14.85,829.90,Dec-2015,C,C5,debt_consolidation,N,Sep-2008,MORTGAGE,785.0,789.0,17.06,Payer
4,10400.0,22.45,289.91,Dec-2015,F,F1,major_purchase,N,Jun-1998,MORTGAGE,695.0,699.0,25.37,Payer


### Data Cleaning

In [23]:
def null_df(df):
    null_percentage = (df.isnull().sum() / len(df)) * 100

    # Convert to DataFrame for better visualization
    null_percentage_df = pd.DataFrame({"Column": null_percentage.index, "Null_Percentage": null_percentage.values})
    null_percentage_df = null_percentage_df[null_percentage_df['Null_Percentage'] > 0.0]

    return null_percentage_df

In [21]:
null_df(df_selected)

,Column,Null_Percentage
0,loan_amnt,0.001460
1,int_rate,0.001460
2,installment,0.001460
3,issue_d,0.001460
4,grade,0.001460
5,sub_grade,0.001460
6,purpose,0.001460
7,debt_settlement_flag,0.001460
8,earliest_cr_line,0.002743
9,home_ownership,0.001460


In [26]:
# Drop all rows with null values in `USER PROFILE`
df_selected = df_selected[df_selected['user_profile'].notnull()]
null_df(df_selected)

,Column,Null_Percentage
8,earliest_cr_line,0.001283
12,dti,0.075686


In [27]:
# Drop all rows with null values in earliest_cr_line because its a `date` column and its not possible to fill it with any value
df_selected = df_selected[df_selected['earliest_cr_line'].notnull()]
# DTI is a continuous variable and can be filled with mean value
df_selected['dti'].fillna(df_selected['dti'].mean(), inplace=True)
null_df(df_selected)

/var/folders/3z/1gg3czrd2ps4jtgnrk_j_nqmgw5xwd/T/ipykernel_81755/863815525.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_selected['dti'].fillna(df_selected['dti'].mean(), inplace=True)


,Column,Null_Percentage
